In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

import numpy as np
import os

import preprocess


In [4]:

# Define shared LSTM layers
def build_shared_lstm(input_shape):
    inputs = layers.Input(shape=(input_shape))
    x = layers.LSTM(32, return_sequences=True)(inputs)
    x = layers.LSTM(4, return_sequences=True)(x)
    # Flatten the output of the LSTM layers
    x = layers.Flatten()(x)
    return inputs, x

# Define task-specific fully connected layers
def task_specific_layers(shared_output):
    x = layers.Dense(32, activation='relu')(shared_output)
    x = layers.Dropout(0.1)(x)
    output = layers.Dense(1)(x)
    return output

# Input shape (for example, time series length of 100 with 10 features)
input_shape = (240, 1)

# Define shared LSTM layers
inputs, shared_output = build_shared_lstm(input_shape)

# Define task-specific outputs
output_task_1 = task_specific_layers(shared_output)  # Task 1 
output_task_2 = task_specific_layers(shared_output)  # Task 2 
# output_task_3 = task_specific_layers(shared_output)  # Task 3 

# Create a model with multiple outputs
model = models.Model(inputs=inputs, outputs=[output_task_1, output_task_2])

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.0005,
                                                             decay_steps=1000,
                                                             decay_rate=0.9)

optimizer = tf.keras.optimizers.Adam(lr_schedule)

# Compile the model with task-specific losses and metrics
model.compile(optimizer=optimizer,
              loss=['mean_squared_error', 'mean_squared_error'])

target_contract = "GMWB"
target_lapse = "nolapse"
target_noiseLevel = "LowNoise"
target_dataSize = 50000
test_size = 0
seed = 22
patience = 50
n_epochs = 3000
batch_size = 4096

cwd = f"../sim_VA/result/{target_contract}/{target_lapse}/"

price = np.load(cwd + f"outerScenarios_{target_contract}_RS_{target_lapse}.npy")
rtn = (price[:, 1:] - price[:, :-1]) / price[:, :-1]

if target_noiseLevel == "LowNoise":
    N = 100
else:
    N = 10

substring = f"hedgingLoss_{target_contract}_RS_{N}_{target_lapse}"
loss_file = [file for file in os.listdir(cwd) if substring in file and os.path.isfile(os.path.join(cwd, file))][0]


X_train, y_train_1, X_test, y_test, y_mean, y_std = preprocess.transform_data(rtn, np.load(cwd + loss_file), True, test_size, seed,
                                                                            part=True, part_size=target_dataSize)

loss = np.load("./GMMB.npy")

_, y_train_2, _, _, _, _ = preprocess.transform_data(rtn, loss, True, test_size, seed,
                                                     part=True, part_size=target_dataSize)


# Fit the model
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)

history = model.fit(X_train, (y_train_1, y_train_2), epochs=n_epochs,
                    batch_size=batch_size,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[callback])

model.save("./multiTask_model_50000_extensive.keras")


Data is split, training and test data has shape:
(50000, 240, 1) (50000,) (50000, 240, 1) (50000,)
Data is split, training and test data has shape:
(50000, 240, 1) (50000,) (50000, 240, 1) (50000,)
Epoch 1/3000
11/11 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - dense_5_loss: 0.9956 - dense_7_loss: 1.0012 - loss: 1.9968 - val_dense_5_loss: 0.9777 - val_dense_7_loss: 0.8657 - val_loss: 1.8933
Epoch 2/3000
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - dense_5_loss: 0.9694 - dense_7_loss: 0.9453 - loss: 1.9147 - val_dense_5_loss: 0.9186 - val_dense_7_loss: 0.7816 - val_loss: 1.7514
Epoch 3/3000
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - dense_5_loss: 0.9023 - dense_7_loss: 0.8363 - loss: 1.7386 - val_dense_5_loss: 0.8012 - val_dense_7_loss: 0.6057 - val_loss: 1.4646
Epoch 4/3000
11/11 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - dense_5_loss: 0.7909 - dense_7_loss: 0.6260 - loss: 1.4169 - val_dense_5_loss: 0.7169 - val_dense_7_loss: 0.4632 - val_loss: 1.2349
Epoch 5/3000
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - dens

KeyboardInterrupt: 